# Setup

In [17]:
import os
azure_endpoint=os.getenv("OPENAI_API_BASE_2")
api_key=os.getenv("OPENAI_API_KEY_2")
api_version=os.getenv("OPENAI_API_VERSION_2")
crt = os.getenv("SSL_CERT_FILE")
crt_2 = os.getenv("REQUESTS_CA_BUNDLE")

print(azure_endpoint, api_key, api_version, crt, crt_2)

https://analitica-dev-oai-02.openai.azure.com/ 7f52a2a2fea840dcb0a8330c0e4aa7ea 2024-02-15-preview C:\Users\dmanriqu\Documents\Configs_OpenAI\bancolombia_certs.crt C:\Users\dmanriqu\Documents\Configs_OpenAI\bancolombia_certs.crt


In [1]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_API_BASE_2"),
    api_key=os.getenv("OPENAI_API_KEY_2"),
    api_version=os.getenv("OPENAI_API_VERSION_2")
)

response = client.chat.completions.create(
    model="gpt-4-cde-aia",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ],
    logprobs=True,
    temperature=0
)

print(response.choices[0].message.content)

ImportError: cannot import name 'AzureOpenAI' from 'openai' (c:\Users\dmanriqu\Python\Python312\enviroments\prueba\Lib\site-packages\openai\__init__.py)

In [2]:
import os
import openai
 
openai.api_type = os.getenv("OPENAI_API_TYPE_2")
openai.api_key = os.getenv("OPENAI_API_KEY_2")
openai.api_base = os.getenv("OPENAI_API_BASE_2")
openai.api_version = os.getenv("OPENAI_API_VERSION_2")

def llm(messages, prompt, stop=["\n"]):
    messages[-1]["content"] = prompt
    response = openai.ChatCompletion.create(
      engine="gpt-35-turbo-16k-cde-aia",
      messages=messages,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["message"]["content"]

In [3]:
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
llm(messages, messages[-1]["content"], stop=[])

'Yes, other Azure AI services also support customer managed keys. Azure provides a range of AI services such as Azure Cognitive Services, Azure Machine Learning, and Azure Bot Service, all of which support customer managed keys for enhanced security and control over data encryption.'

In [4]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="train")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            print("-"*30)
            attempts += 1

# ReAct

In [5]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
print(webthink_examples)


Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions:
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[

In [6]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

messages = []

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples
messages.append({"role": "system", "content": instruction})


def webthink(idx=None, prompt=webthink_examples, to_print=True, messages=messages):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    messages.append({"role": "user", "content": prompt})
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(messages, prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(messages, prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        messages[-1]["content"] = prompt
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [7]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[:2]:
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Question: Thurn and Taxis and Cluedo, are which form of entertainment?
Thought 1: I need to search Thurn and Taxis and Cluedo, and find their form of entertainment.
Action 1: Search[Thurn and Taxis]
Observation 1: The Princely House of Thurn and Taxis (German: Fürstenhaus Thurn und Taxis, [ˈtuːɐ̯n ʔʊnt ˈtaksɪs]) is a family of German nobility that is part of the Briefadel. It was a key player in the postal services in Europe during the 16th century, until the end of the Holy Roman Empire in 1806, and became well known as the owner of breweries and commissioner of several castles. The family has resided in Regensburg since 1748 with their seat at St. Emmeram Castle from 1803. The family is one of the wealthiest in Germany, and the current head of the House is Albert, 12th Prince of Thurn and Taxis..

Thought 2: Thurn and Taxis is a family of German nobility known for their involvement in postal services and ownership of breweries and castles. I need to search Cluedo next to find it